# 회귀나무(Decision Tree Regressor)

- [회귀 나무](https://scikit-learn.org/stable/modules/tree.html#)
  - 출력변수가 연속형일 때의 의사결정 나무
  - 지니 불순도에 대응되는 개념으로 분산(Variance)을 사용하여 노드를 분할함
  - 신규 데이터에 대한 예측은 끝 노드에 도달하고 해당 노드(영역)에 속하는 학습데이터들의 평균값으로 함
  - 분할된 영역에 클래스 데이터가 최대한 많이 존재하도록 각 축으로 영역을 분할하여 생성


- ![수식](img\회귀나무.png)

### 1. 데이터 로드 및 전처리

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib
#한글꺠짐 방지
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [3]:
data=pd.read_csv("./data/Fault_data.csv",encoding="EUC-KR")
data.drop(["자재구분","END_DATE_TIME"],axis=1,inplace=True)
data.dropna(axis=1,inplace=True)

#X,Y 분할
Y=data["Y"].copy()
X=data.drop("Y",axis=1)
X.head(3)

,CHAMBER,TAE.STDDEV0,TAE.MEAN0,TAE.RANGE0,PC.STDDEV0,PC.MEAN0,PC.RANGE0,CROT.STDDEV0,CROT.MEAN0,CROT.RANGE0,...,PC.RANGE12,TAE.STDDEV9,TAE.MEAN9,TAE.RANGE9,CROT.STDDEV9,CROT.MEAN9,CROT.RANGE9,PC.STDDEV9,PC.MEAN9,PC.RANGE9
0,4,0.044554,38.083127,0.254154,0.124753,0.044329,0.439883,0,35,0,...,17.350926,0.032832,35.011650,0.127077,0,35,0,0.405343,45.124841,3.079182
1,2,0.034801,38.052395,0.063538,0.177575,0.342131,0.439883,0,27,0,...,1.466280,0.020858,35.550245,0.063540,0,27,0,1.923012,45.350277,15.004887
2,3,0.030089,38.046407,0.127080,0.137559,0.063983,0.488759,0,15,0,...,13.098728,0.021635,36.013373,0.063538,0,15,0,0.161640,45.071393,0.977516


### 2. 평가 지표 함수

In [4]:
# 평가 지표 출력 함수
def print_all_reg(Y_test,pred):
    print("model의 MSE 값은 {:.3f}".format(mean_squared_error(Y_test,pred)))
    print("model의 MAE 값은 {:.3f}".format(mean_absolute_error(Y_test, pred)))
    print("model의 MAPE 값은 {:.3f}".format(mean_absolute_percentage_error(Y_test, pred)))
    print("model의 r2 값은 {:.3f}".format(r2_score(Y_test, pred)))

### 3. 모델 학습 및 예측

[[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)]  
**sklearn.model_selection.train_test_split**
- **test_size** : float or int, default = 0.25, 정수값일시 test사이즈로 설정하고 싶은 샘플 수 입력
- **train_size** : float or int, default = None
- **random_state** : int, default = None, 랜덤 seed값 설정, 같은 seed 내에선 동일결과 추출 
- **shuffle** : bool, default = True, 데이터셋 무작위 추출, 시계열 데이터와 같이 순차적 추출이 필요한 경우엔 Shuffle = False!
- **stratify** : array-like, default = None, True일시 계층적 샘플링 진행 ([참고](https://www.investopedia.com/terms/stratified_random_sampling.asp))

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=3,shuffle =True)

[[DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)]  

**사전 가지치기**   
: 나무가 완성(Full Tree)되기 전에 특정조건을 만족하는 경우 알고리즘을 중단하는 방법. 하이퍼파라미터로 손쉽게 설정 가능
      
**주요 하이퍼파라미터**  
- **criterion** : {“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}, default=”squared_error”, 클래스 동질성을 측정하는 지표 설정, 기본값 MSE
- **max_depth** : int, default = None, 트리의 최대깊이를 설정. 값이 클수록 모델의 복잡도가 올라간다.
- **min_samples_split** : int or float, default = 2, 자식노드를 분할하는데 필요한 최소 샘플의 수
- **min_samples_leaf** : leaf node에서 필요한 최소 샘풀수이며, 너무 적을 시 과적합 발생
- **max_leaf_nodes** : int, default=None, 최대 leaf node 수 제한
- **max_features** : int, float or {“auto”, “sqrt”, “log2”}, default=None, 각 노드를 분리할 때 사용 할 최대 속성 수

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
# 원하는 파라미터 설정 (파라미터를 추가하려면 위의 함수에도 추가해줘야함.)
params={
    "max_depth":[4,6,8,10,12,14],
    "min_samples_leaf" : [2, 4, 6,10],
    "min_samples_split" : range(2,8,2),
    "max_leaf_nodes": [5,10,15,20]
}

# ParameterGrid 통해서 모든 경우의 수 만들기 
params_list = list(ParameterGrid(params)) # 6x4x3x4 = 총 288개의 경우의수를 담은 리스트 반환

# grid search 진행
score_list = []
for params2 in params_list:
    model_use = DecisionTreeRegressor(random_state = 0,max_depth = params2["max_depth"],min_samples_leaf = params2["min_samples_leaf"],
                                   min_samples_split = params2["min_samples_split"],max_leaf_nodes = params2["max_leaf_nodes"]
                                  )      
    model_use.fit(X_train, Y_train)
    valid_pred = model_use.predict(X_test)
    tem = mean_squared_error(valid_pred,Y_test)
    score_list.append(tem)

# BEST SCORE 계산
best_index= np.argmax(score_list)
print("BEST SCORE", score_list[best_index])
print("BEST PARAMS", params_list[best_index])

BEST SCORE 3144.0509708270733
BEST PARAMS {'max_depth': 6, 'max_leaf_nodes': 10, 'min_samples_leaf': 10, 'min_samples_split': 2}


In [8]:
# best case의 하이퍼파라미터로 의사결정나무 학습

model_use = DecisionTreeRegressor(random_state = 0,max_depth = params_list[best_index]["max_depth"],min_samples_leaf = params_list[best_index]["min_samples_leaf"],
                                   min_samples_split = params_list[best_index]["min_samples_split"],max_leaf_nodes = params_list[best_index]["max_leaf_nodes"]
                                  )      
model_use.fit(X_train, Y_train)
pred_train = model_use.predict(X_train)
pred_test = model_use.predict(X_test)
tem = mean_squared_error(Y_test,pred_test)
print(tem)

#출력
print("Train 데이터 성능")
print_all_reg(Y_train,pred_train)
print(" ")
print("TEST 데이터 성능")
print_all_reg(Y_test,pred_test)


3144.0509708270733
Train 데이터 성능
model의 MSE 값은 1891.915
model의 MAE 값은 26.063
model의 MAPE 값은 0.495
model의 r2 값은 0.485
 
TEST 데이터 성능
model의 MSE 값은 3144.051
model의 MAE 값은 35.296
model의 MAPE 값은 0.929
model의 r2 값은 0.334
